In [1]:
%AddJar https://repo1.maven.org/maven2/com/datastax/spark/spark-cassandra-connector_2.11/2.4.3/spark-cassandra-connector_2.11-2.4.3.jar

Using cached version of spark-cassandra-connector_2.11-2.4.3.jar


In [2]:
import org.apache.spark.{SparkContext, SparkConf}

println("Starting Spark Session...")

val username = "alexey_chernyaev2"
val password = "fuiBpd2S"
val hostname = "10.0.0.5"

val postgreUrl = s"jdbc:postgresql://$hostname:5432"

val conf = new SparkConf()
                    .set("spark.cassandra.connection.host", "10.0.0.5")
                    .set("spark.cassandra.connection.port", "9042")
                    .set("spark.cassandra.auth.username", "alexey.chernyaev2")
                    .set("spark.cassandra.auth.password", password)
                    .set("spark.cassandra.output.consistency.level", "ANY")
                    .set("spark.cassandra.input.consistency.level", "ONE")
    
val sparkSession = SparkSession.builder()
                        .appName("Lab03")
                        .config(conf=conf)
                        .getOrCreate()

println("Spark Session was started")




Starting Spark Session...
Spark Session was started


username = alexey_chernyaev2
password = fuiBpd2S
hostname = 10.0.0.5
postgreUrl = jdbc:postgresql://10.0.0.5:5432
conf = org.apache.spark.SparkConf@1e8866da
sparkSession = org.apache.spark.sql.SparkSession@6f62a97a


org.apache.spark.sql.SparkSession@6f62a97a

# HDFS

In [3]:
val webLogsDF = spark.read.parquet("/labs/laba03/weblogs.parquet")

println(webLogsDF.cache.count)
webLogsDF.printSchema
webLogsDF.show(3, 200, true)

36138
root
 |-- uid: string (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- timestamp: long (nullable = true)
 |    |    |-- url: string (nullable = true)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 uid    | 39d8aded-2111-42ab-9393-37c4f781d030                                                                                                                                                                     
 visits | [[1419590902274, http://go.mail.ru/], [1419590344372, http://thephilanews.com/ru/wawa-welcome-america-begins-with-reading-…-and-the-runway-32987.htm], [1419590203997, http://visitorsworth.com/www.w... 
-RECORD 1---------------------------------------------------------------------------------------------------------------------------------

webLogsDF = [uid: string, visits: array<struct<timestamp:bigint,url:string>>]


[uid: string, visits: array<struct<timestamp:bigint,url:string>>]

In [4]:
import org.apache.spark.sql.functions._

val webLogsStatsDF = webLogsDF
                        .select($"uid", explode($"visits").as("visit"))
                        .withColumn("host", lower(callUDF("parse_url", $"visit.url", lit("HOST"))))
                        .withColumn("domain", regexp_replace(lower($"host"), "www.", ""))
                        .where($"uid".isNotNull)
                        .groupBy($"uid", $"domain").count

println(webLogsStatsDF.cache.count)
webLogsStatsDF.printSchema
webLogsStatsDF.show(3, 200, true)

572694
root
 |-- uid: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- count: long (nullable = false)

-RECORD 0--------------------------------------
 uid    | 3a081c72-5c55-4c7b-bf52-b56201d85b55 
 domain | videograbber.net                     
 count  | 1                                    
-RECORD 1--------------------------------------
 uid    | 3a48922d-4621-455e-ba68-f273b95807fb 
 domain | umeltsi.ru                           
 count  | 4                                    
-RECORD 2--------------------------------------
 uid    | 63046a3d-830e-44b3-b580-a16db013234d 
 domain | intvua.com                           
 count  | 2                                    
only showing top 3 rows



webLogsStatsDF = [uid: string, domain: string ... 1 more field]


[uid: string, domain: string ... 1 more field]

# postgreSql

In [5]:
%AddJar file:///data/home/alexey.chernyaev2/postgresql-42.2.12.jre6.jar

Using cached version of postgresql-42.2.12.jre6.jar


In [6]:
val domainCatsDF = spark.read
                                .format("jdbc")
                                .option("driver", "org.postgresql.Driver")
                                .option("url", s"$postgreUrl/labdata")
                                .option("dbtable", "domain_cats")
                                .option("user", username)
                                .option("password", password)
                                .load()

domainCatsDF.printSchema
domainCatsDF.show()

root
 |-- domain: string (nullable = true)
 |-- category: string (nullable = true)

+--------------------+--------+
|              domain|category|
+--------------------+--------+
|national-lottery....|gambling|
|          rivalo.com|gambling|
|             xoso.me|gambling|
|         betin.co.ke|gambling|
|          skybet.com|gambling|
|              fdj.fr|gambling|
|     minhngoc.net.vn|gambling|
|   hollywoodbets.net|gambling|
|           tipico.de|gambling|
|       betpawa.co.ke|gambling|
|           bet365.it|gambling|
|          bet365.com|gambling|
|loteriasdominican...|gambling|
|            hkjc.com|gambling|
|        sunmaker.com|gambling|
|     williamhill.com|gambling|
|       parimatch.com|gambling|
|        freebitco.in|gambling|
|   mismarcadores.com|gambling|
|      flashscore.com|gambling|
+--------------------+--------+
only showing top 20 rows



domainCatsDF = [domain: string, category: string]


[domain: string, category: string]

In [7]:
val webStatsDF = webLogsStatsDF
                        .join(domainCatsDF, Seq("domain"), "left")
                        .withColumn("web_cat", concat(lit("web_"), regexp_replace(lower($"category"), "[ -]", "_")))
                        .where($"web_cat".isNotNull)
                        .groupBy($"uid")
                        .pivot($"web_cat")
                        .sum("count")

println(webStatsDF.cache.count)
webStatsDF.printSchema
webStatsDF.show(3, 200, true)


27802
root
 |-- uid: string (nullable = true)
 |-- web_arts_and_entertainment: long (nullable = true)
 |-- web_autos_and_vehicles: long (nullable = true)
 |-- web_beauty_and_fitness: long (nullable = true)
 |-- web_books_and_literature: long (nullable = true)
 |-- web_business_and_industry: long (nullable = true)
 |-- web_career_and_education: long (nullable = true)
 |-- web_computer_and_electronics: long (nullable = true)
 |-- web_finance: long (nullable = true)
 |-- web_food_and_drink: long (nullable = true)
 |-- web_gambling: long (nullable = true)
 |-- web_games: long (nullable = true)
 |-- web_health: long (nullable = true)
 |-- web_home_and_garden: long (nullable = true)
 |-- web_internet_and_telecom: long (nullable = true)
 |-- web_law_and_government: long (nullable = true)
 |-- web_news_and_media: long (nullable = true)
 |-- web_pets_and_animals: long (nullable = true)
 |-- web_recreation_and_hobbies: long (nullable = true)
 |-- web_reference: long (nullable = true)
 |-- web_sc

webStatsDF = [uid: string, web_arts_and_entertainment: bigint ... 22 more fields]


[uid: string, web_arts_and_entertainment: bigint ... 22 more fields]

# ES

In [8]:
%AddJar file:///data/home/alexey.chernyaev2/elasticsearch-spark-20_2.11-7.7.0.jar

Using cached version of elasticsearch-spark-20_2.11-7.7.0.jar


In [9]:

val esOptions =
  Map(
    "es.nodes" -> "10.0.0.5:9200",
    "es.batch.write.refresh" -> "false",
    "es.nodes.wan.only" -> "true",
    "es.net.http.auth.user" -> "alexey.chernyaev2",
    "es.net.http.auth.pass" -> password
  )
var visitsDF = spark.read.format("org.elasticsearch.spark.sql").options(esOptions).load("visits*")

println(visitsDF.cache.count)
visitsDF.printSchema
visitsDF.show(3, 200, true)

182540
root
 |-- category: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- item_id: string (nullable = true)
 |-- item_price: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- uid: string (nullable = true)

-RECORD 0------------------------------------------
 category   | Entertainment-equipment              
 event_type | buy                                  
 item_id    | Entertainment-equipment-2            
 item_price | 2529                                 
 timestamp  | 2020-01-01 11:00:00                  
 uid        | 40b29579-e845-45c0-a34d-03630d296a81 
-RECORD 1------------------------------------------
 category   | Everyday-jewelry                     
 event_type | buy                                  
 item_id    | Everyday-jewelry-0                   
 item_price | 4320                                 
 timestamp  | 2020-01-01 11:01:00                  
 uid        | null                                 
-RECORD 2----------

esOptions = Map(es.nodes.wan.only -> true, es.net.http.auth.user -> alexey.chernyaev2, es.net.http.auth.pass -> fuiBpd2S, es.batch.write.refresh -> false, es.nodes -> 10.0.0.5:9200)
visitsDF = [category: string, event_type: string ... 4 more fields]


[category: string, event_type: string ... 4 more fields]

In [13]:
val visitStatsDF = visitsDF
                    .filter($"uid".isNotNull)
                    .withColumn("shop_group", concat(lit("shop_"), regexp_replace(lower($"category"), "[ -]", "_")))
                    .groupBy($"uid", $"shop_group")
                    .count()
println(visitStatsDF.cache.count)
visitStatsDF.printSchema
visitStatsDF.show(3, 200, true)


86507
root
 |-- uid: string (nullable = true)
 |-- shop_group: string (nullable = true)
 |-- count: long (nullable = false)

-RECORD 0------------------------------------------
 uid        | 75fcbe0c-ef8a-452f-8864-ea3a4f376ae2 
 shop_group | shop_computers                       
 count      | 1                                    
-RECORD 1------------------------------------------
 uid        | 9e241ecb-af15-49a7-8fef-516aaae0456d 
 shop_group | shop_cameras                         
 count      | 1                                    
-RECORD 2------------------------------------------
 uid        | 372c086a-c53d-45fd-b947-abc96612030a 
 shop_group | shop_mobile_phones                   
 count      | 1                                    
only showing top 3 rows



visitStatsDF = [uid: string, shop_group: string ... 1 more field]


[uid: string, shop_group: string ... 1 more field]

## Cassandra

In [11]:
//%AddJar https://repo1.maven.org/maven2/com/datastax/spark/spark-cassandra-connector_2.11/2.4.3/spark-cassandra-connector_2.11-2.4.3.jar

lastException: Throwable = null


Name: Syntax Error.
Message: 
StackTrace: 

In [14]:

val tableOpts = Map("table" -> "clients", "keyspace" -> "labdata")
val clients = spark
  .read
  .format("org.apache.spark.sql.cassandra")
  .options(tableOpts)
  .load()
clients.show()

+--------------------+---+------+
|                 uid|age|gender|
+--------------------+---+------+
|075bdfc4-cfa6-4f0...| 39|     F|
|a1096800-f5d8-4f1...| 52|     M|
|594d59f5-22db-4c2...| 39|     M|
|e60a27f7-9c8e-4c6...| 30|     M|
|bf620165-604f-4ea...| 65|     F|
|97fd1ca4-b2b6-4f8...| 20|     F|
|d219a21c-af6d-484...| 39|     M|
|0e38d915-ed29-4ab...| 49|     M|
|9f968029-11c0-429...| 39|     F|
|1784f7ee-f8fa-419...| 19|     F|
|400a54aa-51dd-4e3...| 32|     F|
|1ffe78da-81b1-4ea...| 39|     M|
|58fd46de-2e9d-4b8...| 26|     M|
|d066f4cb-fa3f-4a9...| 30|     F|
|8608a863-1abe-47f...| 26|     M|
|a4404d1c-9d82-451...| 31|     F|
|d226a5bf-287d-47c...| 36|     M|
|2d256dd9-771d-4e8...| 29|     M|
|b9ad76d6-b20f-4e5...| 34|     F|
|eee0b882-a6a8-4f3...| 25|     M|
+--------------------+---+------+
only showing top 20 rows



tableOpts = Map(table -> clients, keyspace -> labdata)
clients = [uid: string, age: int ... 1 more field]


[uid: string, age: int ... 1 more field]

In [16]:
import org.apache.spark.sql.functions.when

val shopStatsDF = clients
                        .withColumn("age_group", when($"age" >= 55, ">=55")
                                              .when($"age" >= 45, "45-54")
                                              .when($"age" >= 35, "35-44")
                                              .when($"age" >= 25, "25-34")
                                              .when($"age" >= 18, "18-24"))
                        .join(visitStatsDF, Seq("uid"), "left")
                        .groupBy($"uid", $"gender", $"age_group")
                        .pivot($"shop_group")
                        .sum("count")
                        .drop($"null")
println(shopStatsDF.cache.count)
shopStatsDF.printSchema
shopStatsDF.show(3, 200, true)


36138
root
 |-- uid: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age_group: string (nullable = true)
 |-- shop_cameras: long (nullable = true)
 |-- shop_clothing: long (nullable = true)
 |-- shop_computers: long (nullable = true)
 |-- shop_cosmetics: long (nullable = true)
 |-- shop_entertainment_equipment: long (nullable = true)
 |-- shop_everyday_jewelry: long (nullable = true)
 |-- shop_house_repairs_paint_tools: long (nullable = true)
 |-- shop_household_appliances: long (nullable = true)
 |-- shop_household_furniture: long (nullable = true)
 |-- shop_kitchen_appliances: long (nullable = true)
 |-- shop_kitchen_utensils: long (nullable = true)
 |-- shop_luggage: long (nullable = true)
 |-- shop_mobile_phones: long (nullable = true)
 |-- shop_shoes: long (nullable = true)
 |-- shop_sports_equipment: long (nullable = true)
 |-- shop_toys: long (nullable = true)

-RECORD 0--------------------------------------------------------------
 uid                       

shopStatsDF = [uid: string, gender: string ... 17 more fields]


lastException: Throwable = null


[uid: string, gender: string ... 17 more fields]

In [17]:
shopStatsDF.show()

+--------------------+------+---------+------------+-------------+--------------+--------------+----------------------------+---------------------+------------------------------+-------------------------+------------------------+-----------------------+---------------------+------------+------------------+----------+---------------------+---------+
|                 uid|gender|age_group|shop_cameras|shop_clothing|shop_computers|shop_cosmetics|shop_entertainment_equipment|shop_everyday_jewelry|shop_house_repairs_paint_tools|shop_household_appliances|shop_household_furniture|shop_kitchen_appliances|shop_kitchen_utensils|shop_luggage|shop_mobile_phones|shop_shoes|shop_sports_equipment|shop_toys|
+--------------------+------+---------+------------+-------------+--------------+--------------+----------------------------+---------------------+------------------------------+-------------------------+------------------------+-----------------------+---------------------+------------+----------

# create DataMart

In [ ]:
val datamartDF = shopStatsDF
                        .join(webStatsDF, Seq("uid"), "left")
                        .orderBy($"uid")
println(datamartDF.cache.count)
datamartDF.printSchema
datamartDF.show(3, 200, true)


In [ ]:
val options = Map(
                 "url" -> s"$postgreUrl/$username",
                 "user" -> username,
                 "password" -> password,
                 "mode" -> "overwrite",
                 "driver" -> "org.postgresql.Driver",
                 "dbtable" -> "clients")

datamartDF.write.format("jdbc").options(options).mode("overwrite").save()